Basically copied from the excellen

In [ ]:
import pystac
import rasterio
from shapely.geometry import Polygon, mapping
import json
import os
from datetime import datetime

In [ ]:
def get_bbox_and_footprint(raster_uri):
    with rasterio.open(raster_uri) as ds:
        bounds = ds.bounds
        bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
        footprint = Polygon([
            [bounds.left, bounds.bottom],
            [bounds.left, bounds.top],
            [bounds.right, bounds.top],
            [bounds.right, bounds.bottom]
        ])

        return (bbox, mapping(footprint))

In [ ]:
def add_item(img_path, name):
    bbox, footprint = get_bbox_and_footprint(img_path)
    item = pystac.Item(id=name,
                 geometry=footprint,
                 bbox=bbox,
                 datetime=datetime.utcnow(),
                 properties={})
    catalog.add_item(item)
    

In [ ]:
catalog = pystac.Catalog(id='20211216_raster_work', description='project')
mydir = r"...\data"
for file in os.listdir(mydir):
    if file.endswith((".tif")): 
        fullfile = os.path.join(mydir, file)
        add_item(fullfile, file)

In [ ]:
catalog.normalize_hrefs(os.path.join(mydir, '20211216_raster_work'))
catalog.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)
catalog.save(catalog_type=pystac.CatalogType.ABSOLUTE_PUBLISHED)

In [ ]:
import json

json_file = r'...\data\20211216_raster_work/catalog.json'

print('Opening JSON file...')
with open(json_file) as f:
    data = json.load(f)
    print('JSON file opened.')

print('Printing data...')

for link in data['links']:
    if link['href'].find('NAME_OF_FILE_YOU_WANT_TO_INSPECT') != -1:
        with open(link['href'], 'rb') as f1:
            new_raster = json.load(f1)
            print(new_raster)